In [ ]:
import sys
import time
import numpy as np
import pandas as pd
import itertools
import plotly.express as px
import plotly.graph_objects as go
import datetime
from tqdm import trange

from sklearn.covariance import GraphicalLasso as sk_GL
from sklearn.covariance import empirical_covariance
from sklearn import set_config
set_config(print_changed_only=False)

from gglasso.solver.single_admm_solver import ADMM_SGL
from gglasso.solver.single_admm_solver import block_SGL
from gglasso.helper.data_generation import time_varying_power_network, group_power_network, sample_covariance_matrix
from gglasso.helper.model_selection import single_grid_search
from benchmarks import models_to_dict, sklearn_time_benchmark, admm_time_benchmark, model_solution, benchmark_parameters
from benchmarks import network_generation, hamming_distance, hamming_dict, time_benchmark,  sparsity_benchmark, dict_shape
from benchmarks import plot_log_distance, plot_scalability, drop_duplicates, dict_to_dataframe, time_dict_to_data_frame
from benchmarks import sk_scaling, single_scaling, block_scaling


from regain.covariance import GraphicalLasso as rg_GL

In [ ]:
S_dict=dict()
X_dict=dict()
Theta_dict=dict()

p_list=[100, 500, 1000, 2500, 5000, 10000]
N_list=[200, 1000, 2000, 5000, 10000, 20000]

print(" Power network generation ".center(40, '-'))

for p, N in zip(p_list, N_list):
    try:
        start = time.time()
        S, X, Theta = network_generation(p, N, K=1, M=10)
        end = time.time()
        print("p: %5d, N : %5d, Time : %5.4f" % (p, N, end-start))
    except:
        print("Power network cannot be generated")
        print("Tip: increase the number of sub-blocks M")
        break

    S_dict[p, N] = S
    X_dict[p, N] = X
    Theta_dict[p, N] = Theta

In [ ]:
print("\n Shape of S_i:", dict_shape(S_dict))
print("\n Shape of X_i:", dict_shape(X_dict))
print("\n Shape of Theta_i:", dict_shape(Theta_dict))

In [ ]:
sk_params, rg_params, admm_params = benchmark_parameters()

## Speed benchmark

In [ ]:
time_dict = dict()
accuracy_dict = dict()
Z_dict = dict()

for X, S in zip(list(X_dict.values()), list(S_dict.values())):
    times, accs, precs = time_benchmark(X=X, S=S, Z_model="sklearn", lambda1=0.1, n_iter=5,
                                        sk_params=sk_params, rg_params=rg_params, admm_params=admm_params)
    
    time_dict.update(times)
    accuracy_dict.update(accs)
    Z_dict.update(precs)

In [ ]:
sparsity = hamming_dict(Theta_dict=Theta_dict, Z_dict=Z_dict, t_rounding=1e-4)

In [ ]:
df = dict_to_dataframe(times=time_dict, acc_dict=accuracy_dict, spars_dict=sparsity)
df = drop_duplicates(df)
df.head()

In [ ]:
fig = plot_log_distance(df, upper_bound=0.01, lower_bound=0.0001)
fig.show()

## Sparsity benchmark

In [ ]:
frames = sparsity_benchmark(df)

In [ ]:
frames[100]

In [ ]:
frames[300]

In [ ]:
frames[1000]

## Scalability benchmark

In [ ]:
lambda1 = 0.01
max_iter=100
n_iter = 5
tol = 1e-4
rtol = 1e-4

In [ ]:
time_dict = dict()
sk = False
rg = False

sk_model = sk_GL(alpha=lambda1, max_iter=max_iter, assume_centered=False)
rg_model = rg_GL(alpha=lambda1, max_iter=max_iter, assume_centered=False)

for X, S in zip(list(X_dict.values()), list(S_dict.values())):
    #sklearn models
    if sk == False:
        sk_list, sk = sk_scaling(X, sk_model, n_iter)
    else:
        print("sklearn kernel has died at p={}".format(len(X[0])))
        
    if rg == False:
        rg_list, rg = sk_scaling(X, rg_model, n_iter)
    else:
        print("regain kernel has died at p={}".format(len(X[0])))
    
    # ADMM models
    Omega_0 = np.eye(len(S))
    try:
        single_list = single_scaling(S, lambda1, Omega_0, max_iter, tol, rtol, n_iter)
    except:
        print("single ADMM kernel has died at p={}".format(len(X[0])))
    try:
        block_list = block_scaling(S, lambda1, Omega_0, max_iter, tol, rtol, n_iter)
    except:
        print("block ADMM kernel has died at p={}".format(len(X[0])))
        
    key = "p_" + str(len(X[0])) + "_N_" + str(len(X))
    
    time_dict["sklearn_" + key] = np.mean(sk_list)
    time_dict["regain_" + key] = np.mean(rg_list)
    time_dict["single-admm_" + key] = np.mean(single_list)
    time_dict["block-admm_" + key] = np.mean(block_list)

In [ ]:
time_df = time_dict_to_data_frame(time_dict)
time_df.head()

In [ ]:
fig = plot_scalability(time_df)
fig.show()